# Use the data from notebook 3 to randomise locations

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt

import geopandas as gpd
import os, bz2, csv, collections, json, sys, pickle, lzma
import numpy as np

import open_cp.network
import open_cp.geometry
import open_cp.sources.chicago
import matplotlib.patches
import shapely.geometry, descartes

import tilemapbase
import pyproj
proj = pyproj.Proj({"init":"epsg:3528"})

In [2]:
data_dir = os.path.join("/media", "disk", "Data")
#data_dir = os.path.join("..", "..", "..", "..", "Data")
#os.listdir(data_dir)
filename = os.path.join(data_dir, "chicago_all.csv.xz")

In [ ]:
with lzma.open(filename, "rt", encoding="utf8") as f:
    reader = csv.reader(f)
    header = next(reader)
    data = []
    for row in reader:
        if row[15] == "":
            continue
        year = int(row[17])
        if year <= 2001:
            continue
        block, x, y = row[3], float(row[15]), float(row[16])
        x /= open_cp.sources.chicago._FEET_IN_METERS
        y /= open_cp.sources.chicago._FEET_IN_METERS
        if x < 10 or y < 10:
            continue
        data.append((block, x, y, row[5], row[6], row[7], row[2], row[0]))

In [ ]:
crime_points = [shapely.geometry.Point(x,y) for _,x,y,*z in data]
xcs = np.asarray([x for _,x,y,*z in data])
ycs = np.asarray([y for _,x,y,*z in data])
len(xcs)

In [ ]:
frame = gpd.read_file("chicago_regions_simple_segments_merged")

### Solve by iterating over points

This seems a bit slower...

In [ ]:
import rtree

index = rtree.index.Index(((i,geo.bounds,None) for i, geo in enumerate(frame.geometry)))

In [ ]:
containing_geo = []
for i, (x, y, pt) in enumerate(zip(xcs, ycs, crime_points)):
    choices = [i for i in index.intersection((x, y)) if frame.geometry[i].intersects(pt)]
    if len(choices) == 0:
        containing_geo.append(None)
    else:
        containing_geo.append(choices[0])
    assert len(choices) <= 1
    if i % 10000 == 0:
        print(i, file=sys.__stdout__)

### Solve by iterating over polys

Think this is faster, though it's still slow...

In [ ]:
len(frame.geometry)

In [ ]:
ii = np.arange(len(data))
to_geo = [None for _ in data]
for index, geo in enumerate(frame.geometry):
    xmin, ymin, xmax, ymax = geo.bounds
    mask = (xcs >= xmin) & (xcs <= xmax) & (ycs >= ymin) & (ycs <= ymax)
    for i in ii[mask]:
        if geo.intersects(crime_points[i]):
            to_geo[i] = index
    if index % 100 == 0:
        print(index, file=sys.__stdout__)

## Check agree

In [ ]:
containing_geo == to_geo

# Load some geometry

Clip to an outline to stop massive polygons on the boundary

In [ ]:
outline = gpd.io.file.read_file(os.path.join(data_dir, "Chicago_Areas.geojson"))
outline = outline.to_crs({"init":"epsg:3528"})
outline = outline.unary_union
outline

### Visualise outliers

There are 3 of them.  We don't care...

In [ ]:
outliers = []
for row, geo_index in zip(data, to_geo):
    if geo_index is None:
        outliers.append((row[1], row[2]))
outliers = np.asarray(outliers)

fig, ax = plt.subplots(figsize=(8,8))
ax.add_patch(descartes.PolygonPatch(outline, fc="none", ec="black", linewidth=1))
ax.scatter(*outliers.T)
ax.set_aspect(1)
outliers.shape

## Buffer and prepare for random assignment

In [ ]:
outline = outline.buffer(50)
geo = [p.buffer(10) for p in frame.geometry]
geo = [g.intersection(outline) for g in geo]

In [ ]:
computed = {"outline" : outline, "geo" : geo, "to_geo" : to_geo, "data" : data}
with lzma.open("chicago_computed_4.pic.xz", "wb") as f:
    pickle.dump(computed, f)

# Resample

Idea is reallocate points to a random place in the containing geometry.

In [3]:
with lzma.open("chicago_computed_4.pic.xz", "rb") as f:
    computed = pickle.load(f)
outline = computed["outline"]
geo = computed["geo"]
to_geo = computed["to_geo"]
data = computed["data"]

In [ ]:
problem_rows = []
for count, (row, i) in enumerate(zip(data, to_geo)):
    if i is None:
        continue
    if geo[i].is_empty:
        print(row)
        problem_rows.append(count)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.add_patch(descartes.PolygonPatch(outline, fc="none", ec="black", linewidth=1))
x, y = 342649.44343521784, 588676.6803623462
d = 10000
ax.scatter(x, y)
ax.set(xlim=[x-d,x+d], ylim=[y-d,y+d])
ax.set_aspect(1)

# Uniform in each cell

In [4]:
point_cache = dict()

def make_point(p, size=200):
    xmin, ymin, xmax, ymax = p.bounds
    xd, yd = xmax-xmin, ymax-ymin
    xx = np.random.random(size) * xd + xmin
    yy = np.random.random(size) * yd + ymin
    pts = shapely.geometry.MultiPoint(np.asarray([xx,yy]).T)
    pts = pts.intersection(p)
    try:
        return [list(pt.coords)[0] for pt in pts]
    except TypeError:
        return [list(pts.coords)[0]]

def fetch(geo_index):
    if geo_index not in point_cache or len(point_cache[geo_index]) == 0:
        point_cache[geo_index] = make_point(geo[geo_index])
    return point_cache[geo_index].pop()

In [5]:
new_data = []
for count, (row, i) in enumerate(zip(data, to_geo)):
    if i is None or geo[i].is_empty:
        continue
    x, y = fetch(i)
    x *= open_cp.sources.chicago._FEET_IN_METERS
    y *= open_cp.sources.chicago._FEET_IN_METERS
    new_row = (row[0], x, y) +  row[3:]
    new_data.append(new_row)
    if count % 5000 == 0:
        print(count, "/", len(data), file=sys.__stdout__)

In [20]:
def write(filename, new_data):
    header = ["BLOCK", "X", "Y", "CRIME", "SUB-TYPE", "LOCATION", "TIMESTAMP", "CASE"]
    with bz2.open(filename, "wt") as f:
        w = csv.writer(f)
        w.writerow(header)
        w.writerows(new_data)

In [7]:
write("chicago_new_data_test.csv.bz2", new_data)

# Uniform subject to max movement

In [19]:
max_movement = 30

new_data = []
for count, (row, i) in enumerate(zip(data, to_geo)):
    if i is None or geo[i].is_empty:
        continue
    xx, yy = row[1], row[2]
    p = shapely.geometry.Point(xx, yy).buffer(max_movement).intersection(geo[i])
    if p.area < 1:
        print("Skipping", row)
        continue
    while True:
        options = make_point(p, 10)
        if len(options) > 0:
            break
    x, y = options[0]
    x *= open_cp.sources.chicago._FEET_IN_METERS
    y *= open_cp.sources.chicago._FEET_IN_METERS
    new_row = (row[0], x, y) +  row[3:]
    new_data.append(new_row)
    if count % 5000 == 0:
        print(count, "/", len(data), file=sys.__stdout__)    

Skipping ('058XX N RIVER RD', 339111.3251484376, 590374.7211080089, 'STALKING', 'SIMPLE', 'PARKING LOT/GARAGE(NON.RESID.)', '07/06/2007 11:00:00 AM', '5646133')
Skipping ('117XX W IRVING PARK RD', 334443.6180978042, 587151.156411163, 'THEFT', 'FROM BUILDING', 'FACTORY/MANUFACTURING BUILDING', '09/30/2007 03:00:00 PM', '5810856')
Skipping ('058XX N RIVER RD', 339111.3251484376, 590374.7211080089, 'CRIMINAL TRESPASS', 'TO LAND', 'CTA PLATFORM', '11/13/2007 05:48:00 PM', '5908727')
Skipping ('007XX W OHARE ST', 333630.1069238366, 592855.7930286146, 'CRIMINAL TRESPASS', 'TO STATE SUP LAND', 'AIRPORT EXTERIOR - SECURE AREA', '01/14/2015 10:15:00 AM', '9940426')
Skipping ('058XX N RIVER RD', 339111.3251484376, 590374.7211080089, 'THEFT', '$500 AND UNDER', 'CTA GARAGE / OTHER PROPERTY', '05/04/2008 01:00:00 PM', '6249350')
Skipping ('042XX N DIVISION ST', 334268.35810341255, 587134.6972116897, 'NARCOTICS', 'POSS: HEROIN(WHITE)', 'STREET', '03/13/2009 02:30:00 PM', '6803263')
Skipping ('005XX 

In [21]:
write("chicago_new_data_test1.csv.bz2", new_data)